In [276]:
import json
import logging
import os
import shutil
from pathlib import Path
from pymongo import MongoClient

# from megatron.data.dataset_utils import get_indexed_dataset_
import sys
sys.path.append('..')

import horovod.torch as hvd
from dotenv import load_dotenv
import torch
import numpy as np
from torch.utils.data import DataLoader, DistributedSampler, RandomSampler
import datasets
from huggingface_hub import hf_hub_download
from sklearn.metrics import f1_score, accuracy_score

from lm_experiments_tools import TrainerArgs
from lm_experiments_tools.trainer_refactor import Trainer
from torch.utils.data import Dataset, DataLoader

In [281]:
model_path = "../runs/t5-small/ilpc-large/lr1e-05_linear_adamw_wd1e-02_512-512_bs128_iters250000_pretrained_2sep_enum/run_1/"

In [286]:
from transformers import T5ForConditionalGeneration, AutoModel

In [289]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [290]:
cpt = torch.load(model_path + 'model_best.pth')

In [293]:
model.load_state_dict(cpt['model_state_dict'])

<All keys matched successfully>

In [129]:
# class KGLMLocalDataset(Dataset):
#     def __init__(self, port, db, collection, neighborhood=True):
#         self.client = MongoClient('localhost', port)
#         self.db_name = db
#         self.collection_name = collection
#         self.collection = self.client[db][collection]
#         self.length = self.client[self.db_name].command("collstats", self.collection_name)['count']
#         self.neighborhood = neighborhood

#     def  __getitem__(self, idx):
#         item = {}
#         doc = self.collection.find_one({'_id': str(idx)})
#         print(doc)
        
#         if self.neighborhood:
#             item["input"] = doc['verbalization']
#         else:
#             verbalization = doc['verbalization']
#             inp = '[SEP]'.join(verbalization.split('[SEP]')[:2])
#             item["input"] = inp
            
#         item["outputs"] = doc['target']
#         return item
        
#     def __len__(self):
#         return self.length

In [237]:
# path = '/home/bulatov/bulatov/datasets/ilpc22/small-valid.csv'
# path = '/home/bulatov/bulatov/datasets/ilpc22/large_2sep/large_test.csv'
path = '/home/bulatov/bulatov/datasets/ilpc22/large_2sep/large_valid.csv'

In [238]:
import pandas as pd

df = pd.read_csv(path)

In [239]:
tails = df.verbalized_tail.apply(lambda x: x.split(' [SEP-2] ')[0])

In [240]:
df['tail_name'] = tails

In [241]:
verb2tail = df.groupby('tail_name').agg({'tail':tuple})
verb2tail['tail'] = verb2tail['tail'].apply(lambda x: np.unique(x))

In [246]:
verb2tail[verb2tail['tail'].apply(len) > 1]

,tail
tail_name,
Benjamin Franklin,"[Q34969, Q4888609]"
Berg,"[Q67165, Q67318]"
Chris Anderson,"[Q2536856, Q706222]"
Forest Lawn Memorial Park,"[Q1302545, Q1437214]"
František Babka,"[Q20057364, Q20057365]"
Giubiasco,"[Q1527847, Q69498]"
Heinrich Hoffmann,"[Q215724, Q57437]"
Jacksonville,"[Q16568, Q685644]"
John Hughes,"[Q311263, Q328225]"


In [243]:
duplicate_tail_ids = verb2tail[verb2tail['tail'].apply(len) > 1]['tail'].apply(list).sum()
duplicate_tail_ids

['Q34969',
 'Q4888609',
 'Q67165',
 'Q67318',
 'Q2536856',
 'Q706222',
 'Q1302545',
 'Q1437214',
 'Q20057364',
 'Q20057365',
 'Q1527847',
 'Q69498',
 'Q215724',
 'Q57437',
 'Q16568',
 'Q685644',
 'Q311263',
 'Q328225',
 'Q1470101',
 'Q171150',
 'Q1132231',
 'Q195274',
 'Q101254',
 'Q54264',
 'Q67942',
 'Q68440',
 'Q68457',
 'Q34713',
 'Q43325',
 'Q330629',
 'Q56760250',
 'Q158869',
 'Q249056',
 'Q501338',
 'Q641',
 'Q773853',
 'Q182625',
 'Q464955',
 'Q128452',
 'Q2006869']

In [244]:
df['tail'].isin(set(duplicate_tail_ids)).mean()

0.003094606542882405

In [234]:
df[df['tail'].isin(set(duplicate_tail_ids))]

,Unnamed: 0,verbalization,head,tail,verbalized_tail,tail_name
58,58,predict [SEP] Henry Lange [SEP-2] German carto...,Q73364,Q64,Berlin [SEP-2] capital and largest city of Ger...,Berlin
435,435,predict [SEP] Los Angeles [SEP-2] county seat ...,Q65,Q821198,Berlin [SEP-2] American synthpop band,Berlin
626,626,predict [SEP] Agnes Bernelle [SEP-2] World War...,Q103132,Q64,Berlin [SEP-2] capital and largest city of Ger...,Berlin
696,696,predict [SEP] Ray Brown [SEP-2] American jazz ...,Q354542,Q1302545,Forest Lawn Memorial Park [SEP-2] cemetery in ...,Forest Lawn Memorial Park
1064,1064,predict [SEP] Franz Gürtner [SEP-2] Government...,Q60420,Q64,Berlin [SEP-2] capital and largest city of Ger...,Berlin
...,...,...,...,...,...,...
19588,19588,predict [SEP] Jean-Antoine de Baïf [SEP-2] Fre...,Q379941,Q641,"Venice [SEP-2] city in northeastern Italy, sit...",Venice
19768,19768,predict [SEP] Gerard Philips [SEP-2] Dutch bus...,Q1351930,Q41795540,The Hague [SEP-2] city in South Holland,The Hague
19772,19772,predict [SEP] Anna Seghers [SEP-2] German writ...,Q57393,Q64,Berlin [SEP-2] capital and largest city of Ger...,Berlin
19990,19990,predict [SEP] Norbert Berkowitz [SEP-2] place...,Q7050297,Q64,Berlin [SEP-2] capital and largest city of Ger...,Berlin


In [156]:
df.iloc[0].verbalization

'predict [SEP] Jane Goldman [SEP-2] English screenwriter, author and producer occupation [SEP] occupation screenwriter [SEP-2] writer who writes for TV, films, comics and games [SEP] occupation writer [SEP-2] person who uses written words to communicate ideas and to produce works of literature [SEP] occupation film producer [SEP-2] person who supervises the overall process, creative and financial, of making a film [SEP] place of birth Hammersmith [SEP-2] district in west London, England [SEP] spouse Jonathan Ross [SEP-2] English television and radio presenter [SEP] inverse of spouse Jonathan Ross [SEP-2] English television and radio presenter [SEP] languages spoken, written or signed English [SEP-2] West Germanic language originating in England with linguistic roots in French, German and Vulgar Latin [SEP]'

In [178]:
from transformers import AutoTokenizer, T5Tokenizer
# tokenizer = AutoTokenizer.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [179]:
tokenizer.add_tokens(['[SEP]', '[SEP-2]'])

2

In [180]:
tokenizer('[SEP]')

{'input_ids': [32100, 1], 'attention_mask': [1, 1]}

In [181]:
tokenizer('I love [SEP-2] cats')

{'input_ids': [27, 333, 32101, 10003, 1], 'attention_mask': [1, 1, 1, 1, 1]}

In [182]:
tokenizer('I love[SEP-2] cats')

{'input_ids': [27, 333, 32101, 10003, 1], 'attention_mask': [1, 1, 1, 1, 1]}

In [184]:
tokenizer.decode([27, 333, 32101, 10003, 1])

'I love [SEP-2] cats</s>'

In [176]:
s = 'Vienna[SEP-2] capital and one of nine States of Austria'

In [177]:
s.split('[SEP-2]')[0].strip()

'Vienna'

In [ ]:
получится сегодня сделать датасет, где инференс

In [166]:
tokenizer('[SIP]')

{'input_ids': [784, 134, 4629, 908, 1], 'attention_mask': [1, 1, 1, 1, 1]}

In [168]:
df.iloc[0].verbalized_tail

'novelist [SEP-2] writer of novels'

In [169]:
df.verbalization[0]

'predict [SEP] Jane Goldman [SEP-2] English screenwriter, author and producer occupation [SEP] occupation screenwriter [SEP-2] writer who writes for TV, films, comics and games [SEP] occupation writer [SEP-2] person who uses written words to communicate ideas and to produce works of literature [SEP] occupation film producer [SEP-2] person who supervises the overall process, creative and financial, of making a film [SEP] place of birth Hammersmith [SEP-2] district in west London, England [SEP] spouse Jonathan Ross [SEP-2] English television and radio presenter [SEP] inverse of spouse Jonathan Ross [SEP-2] English television and radio presenter [SEP] languages spoken, written or signed English [SEP-2] West Germanic language originating in England with linguistic roots in French, German and Vulgar Latin [SEP]'

In [134]:
df.verbalized_tail[0]

'Parkinson disease long-term degenerative neurological disorder that mainly affects movement'

In [132]:
df.`averbalization[0]

'predict [SEP] Ferdy Mayne German actor (1916-1998) cause of death [SEP] inverse of cast member Ben-Hur 1959 American epic historical drama film by William Wyler [SEP] languages spoken, written or signed German West Germanic language [SEP]'

In [101]:
'[SEP]'.join(df.iloc[2].verbalization.split('[SEP]')[:2]) + '[SEP]'

'predict [SEP] Richard Dawkins English ethologist, evolutionary biologist and author influenced by [SEP]'

In [102]:
df.iloc[2].verbalization

'predict [SEP] Richard Dawkins English ethologist, evolutionary biologist and author influenced by [SEP] occupation biologist scientist studying living organisms [SEP] occupation actor person who acts in a dramatic or comic production and works in film, television, theatre, or radio [SEP] occupation essayist person who writes essays [SEP]'

In [300]:
class KGLMLocalDataset(Dataset):
    def __init__(self, path, neighborhood=True, description=True, sep='[SEP]', sep2=' [SEP-2] '):
        self.df = pd.read_csv(path)
        self.neighborhood = neighborhood
        self.description = description
        self.sep = sep
        self.sep2 = sep2

    def  __getitem__(self, idx):
        item = {}
        triplet = self.df.iloc[idx]
        item["input"] = triplet.verbalization
        if not self.neighborhood:
            item["input"] = self.drop_neighborhood(item["input"])
        
        item["outputs"] = triplet['verbalized_tail']
        if not self.description:
            item["outputs"] = self.drop_description(item["outputs"])
            
        return item
        
    def __len__(self):
        return self.df.shape[0]
            
    
    def drop_neighborhood(self, text):
        return self.sep.join(text.split(self.sep)[:2]) + self.sep
    
    
    def drop_description(self, text):
        return text.split(self.sep2)[0]

In [304]:
dataset = KGLMLocalDataset(path, neighborhood=True, description=False)

In [305]:
dataset[0]

{'input': 'predict [SEP] Jane Goldman [SEP-2] English screenwriter, author and producer occupation [SEP] occupation screenwriter [SEP-2] writer who writes for TV, films, comics and games [SEP] occupation writer [SEP-2] person who uses written words to communicate ideas and to produce works of literature [SEP] occupation film producer [SEP-2] person who supervises the overall process, creative and financial, of making a film [SEP] place of birth Hammersmith [SEP-2] district in west London, England [SEP] spouse Jonathan Ross [SEP-2] English television and radio presenter [SEP] inverse of spouse Jonathan Ross [SEP-2] English television and radio presenter [SEP] languages spoken, written or signed English [SEP-2] West Germanic language originating in England with linguistic roots in French, German and Vulgar Latin [SEP]',
 'outputs': 'novelist'}

In [295]:
df.shape

(20358, 6)

In [127]:
len(dataset)

5816

In [123]:
import random
i = random.randint(0, 1000)
dataset[i]

{'input': 'predict [SEP] Magdeburg federated state capital city of Saxony-Anhalt, Germany country [SEP] located in the administrative territorial entity Kingdom of Prussia former German state (1701–1918) [SEP] inverse of place of birth Johann Joachim Schwabe German poet [SEP] inverse of place of birth Georg Gradnauer German politician [SEP] inverse of place of death Michael Maier German physician and alchemist [SEP] member of Climate Alliance organization [SEP]',
 'outputs': 'Holy Roman Empire varying complex of lands that existed from 962 to 1806 in Central Europe'}

In [45]:
dl = DataLoader(df)

In [46]:
gen = iter(dl)

### sacreBLEU

In [198]:
from sacrebleu.metrics import BLEU, CHRF, TER
refs = [ # First set of references
             ['The dog bit the man.', 'It was not unexpected.', 'The man bit him first.'],
             # Second set of references
             ['The dog had bit the man.', 'No one was surprised.', 'The man had bitten the dog.'],
           ]
sys = ['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']

In [201]:
BLEU(args=[])

AttributeError: 'list' object has no attribute 'force'

In [199]:
bleu = BLEU()

bleu.corpus_score(sys, refs)

TypeError: __init__() missing 1 required positional argument: 'args'

In [ ]:
Out[3]: BLEU = 48.53 82.4/50.0/45.5/37.5 (BP = 0.943 ratio = 0.944 hyp_len = 17 ref_len = 18)

In [4]: bleu.get_signature()
Out[4]: nrefs:2|case:mixed|eff:no|tok:13a|smooth:exp|version:2.0.0

In [5]: chrf = CHRF()

In [6]: chrf.corpus_score(sys, refs)
Out[6]: chrF2 = 59.73

### f1 metric

In [185]:
from sklearn.metrics import f1_score, roug

In [247]:
encode_plus_kwargs = {'truncation': True, 'padding': 'longest', 'pad_to_multiple_of': 1}

In [186]:
y = torch.ones(1, 10)
p = torch.ones(1, 10)
p[:, :3] = 0

In [190]:
y = ['eat more of', 'these french pies', 'and drink some tea']
p = ['eat some more of', 'these french pies', 'and drink some tea']

In [272]:
# encoded_y = tokenizer.batch_encode_plus(y, **encode_plus_kwargs, return_tensors='pt')['input_ids']
# encoded_p = tokenizer.batch_encode_plus(p, **encode_plus_kwargs, return_tensors='pt')['input_ids']
encoded_y = tokenizer.batch_encode_plus(y, **encode_plus_kwargs)['input_ids']
encoded_p = tokenizer.batch_encode_plus(p, **encode_plus_kwargs)['input_ids']

In [261]:
# f1_score(encoded_y, encoded_p)

In [267]:
rouge = datasets.load_metric('rouge')

rouge.add_batch(references=encoded_y, predictions=encoded_p)
out_rouge = rouge.compute()

In [269]:
out_rouge['rougeLsum'].

AggregateScore(low=Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091), mid=Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091), high=Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091))

{'rouge1': AggregateScore(low=Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091), mid=Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091), high=Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091)),
 'rouge2': AggregateScore(low=Score(precision=0.6, recall=0.75, fmeasure=0.6666666666666665), mid=Score(precision=0.7333333333333334, recall=0.9166666666666666, fmeasure=0.8148148148148148), high=Score(precision=0.8000000000000002, recall=1.0, fmeasure=0.888888888888889)),
 'rougeL': AggregateScore(low=Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091), mid=Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091), high=Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091)),
 'rougeLsum': AggregateScore(low=Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091), mid=Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090

In [192]:
import datasets

In [197]:
f1 = datasets.load_metric('f1')

f1.add_batch(references=y, predictions=p)

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


ValueError: invalid literal for int() with base 10: 'eat some more of'

In [ ]:
f1.

In [191]:
# f1_score(y, p, average='micro')

0.6666666666666666

In [145]:
f1_score(y, p, average='samples')

0.8235294117647058

In [135]:
# from torch.utils.data import CSVDataset

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [4]:
class Holder:
    def __init__(self):
        pass

In [5]:
args = Holder()
args.target_seq_len = 512
args.input_seq_len = 512
args.input_prefix = ''

In [6]:
encode_plus_kwargs = {'truncation': True, 'padding': 'longest', 'pad_to_multiple_of': 1}

In [7]:
def collate_fn(batch):
    # cut too long strings because they may slow down tokenization
    inputs = [b['input'][:args.input_seq_len * 10] for b in batch]
    print('inputs', inputs)
    if 'outputs' in batch[0]:
        # if we have more than 1 label per example (only in valid) take only one of them
        # to compute loss on valid
        labels = [b['outputs'][0][:args.target_seq_len * 10] for b in batch]
    else:
        labels = [b['output'][:args.target_seq_len * 10] for b in batch]
    if args.input_prefix:
        inputs = [args.input_prefix + inp for inp in inputs]
    features = tokenizer.batch_encode_plus(list(inputs), max_length=args.input_seq_len, return_tensors='pt',
                                           **encode_plus_kwargs)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer.batch_encode_plus(list(labels), max_length=args.target_seq_len, return_tensors='pt',
                                             **encode_plus_kwargs).input_ids
    labels[labels == tokenizer.pad_token_id] = -100
    features['labels'] = labels
    # features['id'] = [b['id'] for b in batch]
    if 'outputs' in batch[0]:
        features['target_text'] = [b['outputs'] for b in batch]
    else:
        features['target_text'] = [b['output'] for b in batch]
    if 'global_attention_mask' in features:
        raise RuntimeError('What global attention mask for Longformer and LongformerEncoder-Decoder should be?')
    return features

In [10]:
DB_PORTPORT = 15015
train_dataset = KGLMDataset(DB_PORT, 'KGLM', 'train', neighborhood=True)

In [11]:
train_dataset.collection.find_one()

{'_id': '0',
 'head': 'Q29387131',
 'relation': 'P31',
 'tail': 'Q5',
 'verbalization': 'predict [SEP] Lalit Kumar Goel instance of [SEP] ',
 'target': 'human'}

In [9]:
train_sampler = RandomSampler(train_dataset)
kwargs = {'pin_memory': True, }
train_dataloader = DataLoader(train_dataset, batch_size=2, sampler=train_sampler,
                              collate_fn=collate_fn, **kwargs)

In [22]:
list(client.list_databases())

[{'name': 'KGLM', 'sizeOnDisk': 5886840832, 'empty': False},
 {'name': 'KGLM-inductive', 'sizeOnDisk': 1347354624, 'empty': False},
 {'name': 'admin', 'sizeOnDisk': 40960, 'empty': False},
 {'name': 'config', 'sizeOnDisk': 73728, 'empty': False},
 {'name': 'local', 'sizeOnDisk': 73728, 'empty': False}]

In [25]:
client = MongoClient('localhost', 15017)
collection = client["KGLM-inductive"]['train']
collection.find_one()

{'_id': ObjectId('637ca5613c82960d48908165'),
 'head': 'Q5142631',
 'relation': 'P159',
 'tail': 'Q3141'}

In [24]:
client = MongoClient('localhost', 15017)
collection = client["KGLM"]['train']
collection.find_one()

{'_id': '0',
 'head': 'Q29387131',
 'relation': 'P31',
 'tail': 'Q5',
 'verbalization': 'predict [SEP] Lalit Kumar Goel instance of [SEP] ',
 'target': 'human'}

In [10]:
# b = train_dataset[10]

In [11]:
gen = iter(train_dataloader)
sample = next(gen)

{'_id': '19599398', 'head': 'Q16325376', 'relation': 'P17', 'tail': 'Q794', 'verbalization': 'predict [SEP] Nazhvan Recreational Complex country [SEP] ', 'target': 'Iran'}
{'_id': '20226282', 'head': 'Q3313042', 'relation': 'P2416', 'tail': 'Q165704', 'verbalization': 'predict [SEP] Miguel Ángel Sancho sports discipline competed in [SEP] sport athletics [SEP] country of citizenship Spain [SEP] instance of human [SEP] place of birth Valencia [SEP]', 'target': 'high jump'}
inputs ['predict [SEP] Nazhvan Recreational Complex country ', 'predict [SEP] Miguel Ángel Sancho sports discipline competed in ']


In [ ]:
train_dataset[1]

In [ ]:
sample['labels']

In [ ]:
tokenizer(sample['target_text'])

In [ ]:
tokenizer.batch_decode(sample['labels'])

In [ ]:
batch = [train_dataset[0]]

In [19]:
inputs = [b['input'][:args.input_seq_len * 10] for b in batch]
if 'outputs' in batch[0]:
    # if we have more than 1 label per example (only in valid) take only one of them
    # to compute loss on valid
    labels = [b['outputs'][:args.target_seq_len * 10] for b in batch]
else:
    labels = [b['output'][:args.target_seq_len * 10] for b in batch]
if args.input_prefix:
    inputs = [args.input_prefix + inp for inp in inputs]
features = tokenizer.batch_encode_plus(list(inputs), max_length=args.input_seq_len, return_tensors='pt',
                                       **encode_plus_kwargs)
with tokenizer.as_target_tokenizer():
    labels = tokenizer.batch_encode_plus(list(labels), max_length=args.target_seq_len, return_tensors='pt',
                                         **encode_plus_kwargs).input_ids
print(labels)
labels[labels == tokenizer.pad_token_id] = -100
features['labels'] = labels
# features['id'] = [b['id'] for b in batch]
if 'outputs' in batch[0]:
    features['target_text'] = [b['outputs'] for b in batch]
else:
    features['target_text'] = [b['output'] for b in batch]
if 'global_attention_mask' in features:
    raise RuntimeError('What global attention mask for Longformer and LongformerEncoder-Decoder should be?')


tensor([[936,   1]])


In [20]:
labels = [b['outputs'][:args.target_seq_len * 10] for b in batch]
tokenizer.batch_encode_plus(labels)

{'input_ids': [[936, 1]], 'attention_mask': [[1, 1]]}

In [21]:
labels

['human']

In [22]:
# tokenizer.batch_decode(labels)

In [23]:
# cpt_folder = "/home/bulatov/bulatov/KGLM/runs/t5-small/lr5e-05_constant_with_warmup_adamw_wd1e-03_512-512_bs64_iters1000000_baseline/run_0/"
# cpt_folder = "/share/home/export/rmt_internship/kglm/lr5e-05_constant_with_warmup_adamw_wd1e-03_512-512_bs64_iters1000000/run_0"
# cpt_folder = "/home/bulatov/bulatov/KGLM/runs/t5-small/lr5e-05_constant_with_warmup_adamw_wd1e-03_512-512_bs32_iters1500000_baseline/run_0"
cpt_folder = "/home/bulatov/bulatov/KGLM/runs/t5-small/lr5e-05_constant_with_warmup_adamw_wd1e-03_512-512_bs64_iters4000000/run_1/"
# cpt_folder = "/home/bulatov/bulatov/KGLM/tests/runs/test_t5_pretrain/"

In [24]:
import os


In [25]:
# !ls /share/home/export/rmt_internship/kglm/

In [26]:
model_cpt = os.path.join(cpt_folder, 'model_best.pth')
config_path = os.path.join(cpt_folder, 'config.json')

In [27]:
cpt = torch.load(model_cpt, map_location='cpu')

In [28]:
from transformers import T5ForConditionalGeneration, T5Config, AutoConfig

In [29]:
config = T5Config(config_path)

In [38]:
model_cfg = T5Config.from_pretrained(config_path)
# model_cfg

You are using a model of type encoder-decoder to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [36]:
model_cfg = AutoConfig.from_pretrained('t5-small')
# model_cfg

T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to German: "
    },
    "translation_en_to_fr": {
      "early_stopping": true,

In [34]:
model_cpt = os.path.join(cpt_folder, 'model_best.pth')
config_path = os.path.join(cpt_folder, 'config.json')

# model_cfg = AutoConfig.from_pretrained('t5-small')
model_cfg = T5Config.from_pretrained(config_path)
model = T5ForConditionalGeneration(config=model_cfg)

cpt = torch.load(model_cpt, map_location='cpu')
model.load_state_dict(cpt['model_state_dict'])

You are using a model of type encoder-decoder to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


<All keys matched successfully>

In [32]:
model.load_state_dict(cpt['model_state_dict'])

<All keys matched successfully>